In [5]:
from tensorflow import keras
import os 
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import random
import tensorflow as tf

class HandLandmark:
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    custom_landmark_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=4) 
    custom_connection_spec = mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2)

    x_min, y_min = float('inf'), float('inf')  
    x_max, y_max = float('-inf'), float('-inf')

    landmark_colors = {
    0: (255, 0, 0),    
    1: (255, 165, 0),  # Thumb
    2: (255, 165, 0),
    3: (255, 165, 0),
    4: (255, 165, 0),
    5: (0, 255, 0),    # Index
    6: (0, 255, 0),
    7: (0, 255, 0),
    8: (0, 255, 0),
    9: (0, 255, 255),  # Middle
    10: (0, 255, 255),
    11: (0, 255, 255),
    12: (0, 255, 255),
    13: (255, 0, 255), # Ring
    14: (255, 0, 255),
    15: (255, 0, 255),
    16: (255, 0, 255),
    17: (0, 0, 255),   # Pinky
    18: (0, 0, 255),
    19: (0, 0, 255),
    20: (0, 0, 255)
    }

    def fit(self, frame:np.ndarray):
        self.frame = frame

    def co_ordinates_finder(self, results) -> tuple:
        """Args: results (process result from mediapipe model)
            Return: tuple with co-ordinates (x_min, x_max, y_min, y_max)"""
        output_image = np.zeros_like(self.frame)
        for ind, hand_landmarks in enumerate(results.multi_hand_landmarks):
            self.mp_drawing.draw_landmarks(
                image = output_image,
                landmark_list = hand_landmarks,
                connections = self.mp_hands.HAND_CONNECTIONS,
                landmark_drawing_spec = None,
                connection_drawing_spec = self.custom_connection_spec)
        
            x_min, y_min = float('inf'), float('inf')  
            x_max, y_max = float('-inf'), float('-inf')
            
            for idx, landmark in enumerate(hand_landmarks.landmark):
                h, w, _ = self.frame.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(self.frame, (cx, cy), 5, self.landmark_colors.get(idx, (255, 255, 255)), -1)
                cv2.circle(output_image, (cx, cy), 5, self.landmark_colors.get(idx, (255, 255, 255)), -1)
                x_min = min(x_min, cx)
                y_min = min(y_min, cy)
                x_max = max(x_max, cx)
                y_max = max(y_max, cy)
                
            co_ordinates = (x_min, x_max, y_min, y_max)
        return output_image, co_ordinates
        
    def landmark_result(self) -> tuple:
        """Args: image
            Returns: (image, results)"""
        self.frame = cv2.cvtColor(cv2.flip(self.frame, 1), cv2.COLOR_BGR2RGB)
        self.frame.flags.writeable = False
        results = self.hands.process(self.frame)
        self.frame.flags.writeable = True
        self.frame = cv2.cvtColor(self.frame, cv2.COLOR_RGB2BGR)
        return results
    
    def process_image(self) -> np.ndarray:
        results =  self.landmark_result()
        if results.multi_hand_landmarks:
            output_image, co_ordinates = self.co_ordinates_finder(results=results)
        else:
            co_ordinates = None
            output_image = self.frame
        return output_image, co_ordinates
        
    def frame_process(self) -> dict:
        """Args: frame (image)
            Return: processed_image (with landmarks) + original image + x_min, x_max, y_min, y_max """
        self.hands = self.mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)
        output_image, co_ordinates = self.process_image()
        self.hands.close()
        return {'co_ordinates':co_ordinates, 'output_image':output_image, 'original_image':self.frame}

In [19]:
TEST_PATH = "Data\\Test"
FOLDER_PATHS = [os.path.join(TEST_PATH, str(i)) for i in range(10)]

recognizer = HandLandmark()
capture = False
cp = cv2.VideoCapture(0)
while True:
    try:
        success, frame = cp.read()

        if not success:
            print("Frame was not succeed")
            continue
            
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        if cv2.waitKey(10) & 0xFF == ord('c'):
            capture = True
    
        if cv2.waitKey(10) & 0xFF == ord('s'):
            capture = False
        
        recognizer.fit(frame)
        results = recognizer.frame_process() 
    
        if results['co_ordinates'] is not None:
            x_min, x_max, y_min, y_max = results['co_ordinates']
            cropped_image = results['output_image'][y_min:y_max, x_min:x_max]
            cropped_image_resized = cv2.resize(cropped_image, (200, 200), cv2.INTER_AREA)
            if capture:
                DESTINATION_PATH = FOLDER_PATHS[9] + '\\' + str(random.randint(-1909090, 12312312)) + '.jpg'
                cv2.imwrite(DESTINATION_PATH, cropped_image_resized)
                print(DESTINATION_PATH)
            cv2.imshow('cropped_image', cropped_image_resized)
        cv2.imshow("result", results['original_image'])
    except Exception as e:
        print(e)
    
cv2.destroyAllWindows()

Data\Test\9\6606085.jpg
Data\Test\9\5223025.jpg
Data\Test\9\11208990.jpg
Data\Test\9\1648269.jpg
Data\Test\9\10581992.jpg
Data\Test\9\2564668.jpg
Data\Test\9\-1837476.jpg
Data\Test\9\7032118.jpg
Data\Test\9\7168503.jpg
Data\Test\9\10958017.jpg
Data\Test\9\1825602.jpg
Data\Test\9\-1180301.jpg
Data\Test\9\10702518.jpg
Data\Test\9\-257703.jpg
Data\Test\9\11432913.jpg
Data\Test\9\11874483.jpg
Data\Test\9\4084480.jpg
Data\Test\9\-407795.jpg
Data\Test\9\7851081.jpg
Data\Test\9\2973254.jpg
Data\Test\9\6594981.jpg
Data\Test\9\5707326.jpg
Data\Test\9\2112618.jpg
Data\Test\9\3648887.jpg
Data\Test\9\939282.jpg
Data\Test\9\-1239530.jpg
Data\Test\9\1132643.jpg
Data\Test\9\6675984.jpg
Data\Test\9\1744479.jpg
Data\Test\9\1911298.jpg
Data\Test\9\555312.jpg
Data\Test\9\11259797.jpg
Data\Test\9\-471897.jpg
Data\Test\9\1963175.jpg
Data\Test\9\3206587.jpg
Data\Test\9\2518913.jpg
Data\Test\9\1338843.jpg
Data\Test\9\1148300.jpg
Data\Test\9\7318185.jpg
Data\Test\9\6564990.jpg
Data\Test\9\-935153.jpg
Data\Tes